In [11]:
import numpy as np, pandas as pd, glob
from matplotlib import pyplot as plt
%matplotlib inline

In [20]:
gtpath = '/home/croth/Downloads/B3502/GENOTYPE/GT/AE017*_genotypes.csv.gz'
dppath = '/home/croth/Downloads/B3502/GENOTYPE/DP/AE017*_depths.csv.gz'
inpath = '/home/croth/Downloads/B3502/GENOTYPE/INFO/AE017*_info_cols.csv.gz'
rapath = '/home/croth/Downloads/B3502/GENOTYPE/AF/AE017*_allele_ratios.csv.gz'

geno_path = sorted(glob.glob(gtpath))
dept_path = sorted(glob.glob(dppath))
info_path = sorted(glob.glob(inpath))
rati_path = sorted(glob.glob(rapath))

## Define ftn for loading dataframes
def makedf(paths,sep='_',fn = 1):
    df = []
    for d in paths:
        
        temp = pd.read_csv(d,index_col=0)
        temp['Chrom'] = int(d.split(sep)[fn])
        temp['Oldix'] = temp.index
        df.append(temp)
        
    return pd.concat(df).reset_index(drop=True)

In [21]:
geno = makedf(geno_path)

In [14]:
samples = ['Stock1','Stock3','Stock4','CF830']

In [15]:
indifix = (geno[samples].sum(axis=1) == 0)
np.sum(indifix)

118634

In [16]:
refdif = (geno[samples].sum(axis=1)==len(samples))
np.sum(refdif)

290

In [17]:
checkdif = geno[(~indifix) & (~refdif)].index

In [18]:
depth = makedf(dept_path)

In [19]:
covx = 10
covixdif = depth[(depth[samples].T.min() >= covx) & 
                 (depth.index.isin(checkdif))]

In [10]:
ratio = makedf(rati_path)

In [11]:
ratx = 0.8
checkpass = ratio[(ratio[samples].T.min() >= ratx) & 
                  (ratio.index.isin(covixdif.index))].index
len(checkpass)

2155

In [12]:
geno[(geno.index.isin(checkpass))].head()

,CF830,JEC21,JEC20,Stock7,Stock6,Stock1,Stock3,Stock5,Stock2,Stock4,Chrom,Oldix
2935,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2935
4644,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1,4644
5261,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1,5261
7282,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,7282
8773,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,8773


In [13]:
depth[(depth.index.isin(checkpass))].head()

,CF830,JEC21,JEC20,Stock7,Stock6,Stock1,Stock3,Stock5,Stock2,Stock4,Chrom,Oldix
2935,231.0,251.0,273.0,86.0,222.0,180.0,231.0,219.0,188.0,235.0,1,2935
4644,105.0,92.0,143.0,43.0,91.0,78.0,69.0,47.0,51.0,66.0,1,4644
5261,207.0,224.0,246.0,64.0,68.0,40.0,59.0,115.0,46.0,44.0,1,5261
7282,121.0,90.0,115.0,29.0,54.0,48.0,20.0,48.0,40.0,29.0,1,7282
8773,255.0,241.0,271.0,82.0,169.0,99.0,162.0,134.0,101.0,122.0,1,8773


In [14]:
ratio[(ratio.index.isin(checkpass))].head()

,CF830,JEC21,JEC20,Stock7,Stock6,Stock1,Stock3,Stock5,Stock2,Stock4,Chrom,Oldix
2935,0.995690,0.996032,0.996350,0.988506,0.995516,0.994475,0.995690,0.995455,0.994709,0.995763,1,2935
4644,0.990566,0.989247,0.993056,0.977273,0.989130,0.987342,0.985714,0.979167,0.980769,0.970149,1,4644
5261,0.995192,0.995556,0.995951,0.984615,0.985507,0.975610,0.983333,0.991379,0.978723,0.977778,1,5261
7282,0.983607,0.989011,0.991379,0.966667,0.945455,0.979592,0.952381,0.979592,0.975610,0.966667,1,7282
8773,0.996094,0.995868,0.996324,0.987952,0.994118,0.980000,0.987730,0.992593,0.990196,0.991870,1,8773


In [24]:
info = makedf(info_path)

In [26]:
info[(info.Pos==1430787)]

,Seqid,Pos,Qual,Callrate,Nallele,Alleles,Maxlen,Minlen,Depth,Type,Chrom,Oldix
7282,AE017341.1,1430787,495.554,1.0,1,G.T,1,1,595,snp,1,7282


In [28]:
geno.loc[7282]

CF830        0.0
JEC21        0.0
JEC20        0.0
Stock7       0.0
Stock6       0.0
Stock1       0.0
Stock3       1.0
Stock5       0.0
Stock2       0.0
Stock4       0.0
Chrom        1.0
Oldix     7282.0
Name: 7282, dtype: float64

In [16]:
tempg = geno[(geno.index.isin(checkpass))][samples+['Oldix']]

In [17]:
samples

['Stock1', 'Stock3', 'Stock4', 'CF830']

In [18]:
sample_names = ['B3502_A1','B3502_B1','B3502_B7','CF830']
tempg.columns = sample_names + ['Oldgeneix']

In [19]:
parentgt = pd.concat([info[(info.index.isin(checkpass))],
                      tempg],axis=1)

In [28]:
np.sum(np.sum(parentgt[sample_names]>0,axis=1) == len(sample_names))

0

In [20]:
parentgt.head()

,Seqid,Pos,Qual,Callrate,Nallele,Alleles,Maxlen,Minlen,Depth,Type,Chrom,Oldix,B3502_A1,B3502_B1,B3502_B7,CF830,Oldgeneix
2935,AE017341.1,530127,15706.400,1.0,1,C.G,1,1,2120,snp,1,2935,0.0,0.0,0.0,1.0,2935
4644,AE017341.1,884619,11033.600,1.0,1,G.A,1,1,790,snp,1,4644,1.0,1.0,1.0,0.0,4644
5261,AE017341.1,1032506,11176.400,1.0,1,T.C,1,1,1121,snp,1,5261,1.0,1.0,1.0,0.0,5261
7282,AE017341.1,1430787,495.554,1.0,1,G.T,1,1,595,snp,1,7282,0.0,1.0,0.0,0.0,7282
8773,AE017341.1,1694543,36510.300,1.0,1,G.C,1,1,1639,snp,1,8773,1.0,1.0,1.0,0.0,8773


In [21]:
for i,j in parentgt.iterrows():
    if j.Oldix != j.Oldgeneix:
        print(i)

In [22]:
parentgt.to_csv('/home/croth/Self-Filamentation_B3502_progeny/GENOTYPE/B3502_stock_variants.csv.gz')

In [23]:
assert 1 == 0

SyntaxError: invalid syntax (<ipython-input-23-8f425085b44a>, line 1)

In [ ]:
segsix = []
covx = 6
ratx = .8
for s in samples:
    
    genodifix = geno[(geno[s]!=0) & (geno[samples].T.sum()!=len(samples))].index.tolist()
    
    depthix = depth[(depth[s]>=covx) & (depth.index.isin(genodifix))].index.tolist()
    
    ratioix = ratio[(ratio[s]>=ratx) & (ratio.index.isin(depthix))].index.tolist()
    
    infoix = info[(info.index.isin(ratioix)) & (info.Callrate==1)].index.tolist()
    
    segsix.append((s,infoix))

In [ ]:
segix = np.unique(np.concatenate([s for (a,s) in segsix]))
len(segix)

In [ ]:
info.loc[segix,:]

In [ ]:
geno.loc[checkpass.index,samples]

In [ ]:
len(segix)

In [ ]:
len(checkpass.index)

In [ ]:
np.sum([a in segix for a in checkpass.index]) == len(checkpass.index)